# 特征转换（Feature Transformation）

在这个笔记本中，我们将原始文本转换为经过分词的输入，这些输入可以被HuggingFace训练脚本使用。将这个分词和提示创建的步骤与训练过程分开是非常重要的，因为这样可以根据每个步骤的需求选择最高效的计算资源。例如，在工作流程的准备阶段，通常使用成本较低的CPU处理器效果最佳，而在模型训练阶段，使用成本较高的GPU实例效果最佳。

重要的是，在准备数据的分词和提示创建步骤中，使用低成本的CPU处理器可以节省计算成本。而对于模型训练这样需要更多计算资源的任务，则使用成本较高的GPU实例更为适合。GPU对于训练深度学习模型非常有效，可以显著加速训练过程。

通过将分词和提示创建步骤与训练步骤分开，您可以更高效地分配计算资源，充分利用低成本的CPU处理器和高成本的GPU实例，优化整个工作流程，并在可能的情况下降低成本。

![Pipeline](./img/generative_ai_pipeline_rlhf_plus.png)

<a name='1'></a>
## Set up Kernel and Required Dependencies

First, check that the correct kernel is chosen.

<img src="img/kernel_set_up.png" width="300"/>

You can click on that to see and check the details of the image, kernel, and instance type.

<img src="img/w3_kernel_and_instance_type.png" width="600"/>

In [2]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=33229983744, available=27910823936, percent=16.0, used=4949753856, free=400433152, active=3906199552, inactive=25742184448, buffers=1634304, cached=27878162432, shared=1200128, slab=2496040960)


In [3]:
%store -r setup_dependencies_passed

In [4]:
try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [5]:
# 导入 transformers 库中的 AutoTokenizer。这个库主要用于自然语言处理，并且 AutoTokenizer 可以自动地处理各种预训练的模型的分词操作。
from transformers import AutoTokenizer

# 导入 datasets 库中的 load_dataset 和 DatasetDict。datasets 是一个用于处理和管理大规模数据集的库，load_dataset 能够加载各种格式的数据集，而 DatasetDict 则是一种用于处理多个数据集的高级字典。
from datasets import load_dataset, DatasetDict

# 导入 os 库。os 库提供了许多与操作系统交互的函数，例如读写文件、管理目录路径等。
import os

# 导入 time 库。time 库提供了各种与时间有关的函数，例如获取当前时间、使程序暂停等。
import time

## 确保基础数据集已下载

In [6]:
# 检查 './data-summarization/' 目录是否存在。os.path.isdir 函数用于检查指定的路径是否为一个目录。
if os.path.isdir('./data-summarization/'):
    # 如果目录已经存在，打印 'Dataset already downloaded'，表示数据集已经下载。
    print('Dataset already downloaded')
else:
    # 如果目录不存在，那么开始下载和处理数据集。
    
    # 从 datasets 库导入 concatenate_datasets 函数，该函数用于将多个数据集合并为一个数据集。
    from datasets import concatenate_datasets
    
    # 使用 load_dataset 函数加载名为 "knkarthick/dialogsum" 的数据集。
    dataset = load_dataset("knkarthick/dialogsum")
    
    # 使用 concatenate_datasets 函数将数据集的 'train'、'test' 和 'validation' 部分合并为一个数据集。
    dataset = concatenate_datasets([dataset['train'], dataset['test'], dataset['validation']])
    
    # 使用 shell 命令 'mkdir' 创建一个名为 'data-summarization' 的目录。
    !mkdir data-summarization
    
    # 使用 train_test_split 函数将合并后的数据集分割为训练集和测试集，分割比例为 50%，并设置随机种子为 1234 以确保结果的可复现性。
    dataset = dataset.train_test_split(0.5, seed=1234)
    
    # 将测试集保存为 CSV 文件，文件名为 'dialogsum-1.csv'，并放在 'data-summarization' 目录下。设置参数 index=False 表示不保存行索引。
    dataset['test'].to_csv('./data-summarization/dialogsum-1.csv', index=False)
    
    # 将训练集保存为 CSV 文件，文件名为 'dialogsum-2.csv'，并放在 'data-summarization' 目录下。设置参数 index=False 表示不保存行索引。
    dataset['train'].to_csv('./data-summarization/dialogsum-2.csv', index=False)

Dataset already downloaded


## 加载分词器（Tokenizer）和HuggingFace数据集（Dataset）。

In [7]:
%store -r model_checkpoint

In [8]:
# 使用 AutoTokenizer 的 from_pretrained 方法加载预训练的分词器。model_checkpoint 是预训练模型的名称或路径，这里是'google/flan-t5-base'。
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# 使用 load_dataset 函数加载 './data-summarization/' 路径下的数据集。
dataset = load_dataset('./data-summarization/')

# 打印加载的数据集。这将显示数据集的基本信息，包括其包含的样本数、特征名称等。
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/data-summarization-55f5a7efa2dfb458/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 14460
    })
})

## 浏览Prompt样例

In [9]:
# 设置索引值为 0，这将用于从数据集中选取样本。
idx = 0

# 从 'train' 部分的第 idx 个样本中获取 'dialogue' 和 'summary' 字段的值。'dialogue' 字段包含了对话的内容，'summary' 字段包含了对话的摘要。
diag = dataset['train'][idx]['dialogue']
baseline_human_summary = dataset['train'][idx]['summary']

# 构造模型的输入提示。输入提示包括对话的内容和一个摘要的开头，后面的模型将生成摘要的剩余部分。
prompt = f'Summarize the following conversation.\n\n{diag}\n\nSummary:'

# 使用预训练的分词器将输入提示转换为模型的输入格式。return_tensors="pt" 表示返回 PyTorch 的张量格式。
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# 打印输入提示和人工总结的摘要。
print(f'Prompt:\n--------------------------\n{prompt}\n--------------------------')
print(f'Baseline human summary : {baseline_human_summary}')

Prompt:
--------------------------
Summarize the following conversation.

#Person1#: If we employ you, what starting salary would you expect?
#Person2#: I'd like to start at 3000 yuan a month.
#Person1#: I think your background and experience are worth the compensation.
#Person2#: Does it include bonuses?
#Person1#: No, there are annual bonuses, one week paid vacation a year, and health insurance.
#Person2#: Very good.

Summary:
--------------------------
Baseline human summary : #Person1# agrees #Person2#'s starting monthly salary would be 3000 yuan and tells #Person2# about other benefits.


## 对数据集进行分词处理（Tokenize the Dataset）

In [10]:
"""
tokenize_function 函数将数据集的每个样本转换为模型的输入格式，然后使用这个函数处理整个数据集。
"""

# 定义一个名为 tokenize_function 的函数，该函数接受一个样本作为输入，然后返回处理后的样本。
def tokenize_function(example):
    # 构造模型的输入提示。输入提示包括对话的内容和一个摘要的开头，后面的模型将生成摘要的剩余部分。
    prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    inp = [prompt + i + end_prompt for i in example["dialogue"]]

    # 使用预训练的分词器将输入提示转换为模型的输入格式，并保存到 'input_ids' 字段。
    # inp是需要被处理的输入文本。
    # padding="max_length"：如果输入的文本长度小于tokenizer的最大长度，那么它将添加特定的填充符号以达到最大长度。这样可以确保所有的输入都有相同的长度，以便能够在神经网络中进行处理。
    # truncation=True：如果输入的文本长度大于tokenizer的最大长度，那么它将截断文本以适应最大长度。
    # return_tensors="pt"：这表示返回的数据应该是PyTorch张量。这对于将数据输入到PyTorch模型中是必要的。
    # .input_ids：这是从tokenizer返回的数据中提取'input_ids'的方法。'input_ids'是一个包含每个输入符号的数字表示的列表。
    # 这个数字表示是通过预先定义的词汇表来确定的，每个数字对应词汇表中的一个单词或符号。
    example['input_ids'] = tokenizer(inp, padding="max_length", truncation=True, return_tensors="pt").input_ids

    # 使用预训练的分词器将摘要转换为模型的输入格式，并保存到 'labels' 字段。
    # 设置 padding="max_length" 和 truncation=True 表示如果输入的长度超过模型的最大长度，那么将其截断到最大长度。
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    # 返回处理后的样本。
    return example

# 使用 map 函数和 tokenize_function 函数处理整个数据集。设置 batched=True 表示对数据集的一批样本进行并行处理。
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 使用 remove_columns 函数移除不需要的列。这些列包括 'id'、'topic'、'dialogue' 和 'summary'。
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map:   0%|          | 0/14460 [00:00<?, ? examples/s]

详细地分析一下批处理过程中的 tokenize_function 函数。为了简化讨论，假设我们的批次大小是2，也就是我们一次处理两个样本。

**步骤1：接收批次数据**

首先，函数 tokenize_function 会接收一个包含批次数据的字典，该字典的键是特征名称（例如，"dialogue"和"summary"），值是一个列表，列表的长度等于批次大小。例如，这个字典可能如下所示：

```
example = {
    "dialogue": [
        "Person 1: Hi, how are you?\nPerson 2: I'm good, thanks. How about you?",
        "Person 1: What's the weather like today?\nPerson 2: It's sunny and warm."
    ],
    "summary": [
        "Person 1 and Person 2 greeted each other and asked about each other's well-being.",
        "Person 1 asked about the weather and Person 2 responded it's sunny and warm."
    ]
}
```


**步骤2：构建模型的输入提示**

然后，该函数会为每个对话创建一个输入提示，该提示包含对话的内容和一个摘要的开头。这个过程是通过列表推导完成的：
```
prompt = 'Summarize the following conversation.\n\n'
end_prompt = '\n\nSummary: '
inp = [prompt + i + end_prompt for i in example["dialogue"]]
```

我们得到的 inp 列表如下：
```
[
    "Summarize the following conversation.\n\nPerson 1: Hi, how are you?\nPerson 2: I'm good, thanks. How about you?\n\nSummary: ",
    "Summarize the following conversation.\n\nPerson 1: What's the weather like today?\nPerson 2: It's sunny and warm.\n\nSummary: "
]
```

**步骤3：使用分词器处理输入提示和摘要**

接下来，我们使用分词器将输入提示和摘要转换为模型的输入格式：
```
example['input_ids'] = tokenizer(inp, padding="max_length", truncation=True, return_tensors="pt").input_ids
example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
```
这两行代码会将输入提示和摘要分别转换为模型的输入 ID，并添加到 example 字典中。分词器会处理文本的长度，如果文本的长度超过模型的最大长度，那么将其截断到最大长度。

**步骤4：返回处理后的批次数据**

最后，函数返回处理后的批次数据：
```
return example
```
这个 example 字典可能如下所示：
```
{
    'dialogue': [
        "Person 1: Hi, how are you?\nPerson 2: I'm good, thanks. How about you?",
        "Person 1: What's the weather like today?\nPerson 2: It's sunny and warm."
    ],
    'summary': [
        "Person 1 and Person 2 greeted each other and asked about each other's well-being.",
        "Person 1 asked about the weather and Person 2 responded it's sunny and warm."
    ],
    'input_ids': tensor([[  101,  2595,  4931,  ...,     0,     0,     0],
                         [  101,  2595,  4931,  ...,     0,     0,     0]]),
    'labels': tensor([[ 101, 2023, 2003, ...,    0,    0,    0],
                      [ 101, 2023, 2003, ...,    0,    0,    0]])
}
```
在这个字典中，'input_ids' 和 'labels' 是两个二维张量，第一维度是批次大小（在这个例子中，批次大小为2），第二维度是序列长度。值是单词在词典中的索引。

## 将预处理步骤封装成一个可重复使用的函数。

In [12]:
# 定义一个函数用于对数据集样本进行标记（tokenization）
def tokenize_function(example):
    # 定义输入的提示语
    prompt = 'Summarize the following conversation.\n\n'
    # 定义输入的结束提示语
    end_prompt = '\n\nSummary: '
    # 对每个对话进行处理，加上提示和结束提示
    inp = [prompt + i + end_prompt for i in example["dialogue"]]
    # 对处理后的对话进行标记，将结果存入'input_ids'
    example['input_ids'] = tokenizer(inp, padding="max_length", truncation=True, return_tensors="pt").input_ids
    # 对摘要进行标记，将结果存入'labels'
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    # 返回处理后的样本
    return example

# 定义一个函数用于对数据集进行预处理（包括划分数据集和标记）
def transform_dataset(input_data,
                      output_data,
                      huggingface_model_name,
                      train_split_percentage,
                      test_split_percentage,
                      validation_split_percentage,
                      ):

    # 加载原始数据集
    dataset = load_dataset(input_data)
    print(f'Dataset loaded from path: {input_data}\n{dataset}')
    
    # 加载标记器
    print(f'Loading the tokenizer for the model {huggingface_model_name}')
    tokenizer = AutoTokenizer.from_pretrained(huggingface_model_name)
    
    # 划分训练集、测试集和验证集
    train_testvalid = dataset['train'].train_test_split(1 - train_split_percentage, seed=1234)
    test_valid = train_testvalid['test'].train_test_split(test_split_percentage / (validation_split_percentage + test_split_percentage), seed=1234)
    # 创建一个数据集字典（DatasetDict），其中包括训练集、测试集和验证集。
    # DatasetDict 是 Hugging Face 的 datasets 库中的一个类，它可以用于存储和操作多个数据集。
    # 这里创建的 train_test_valid_dataset 就包括了训练集、测试集和验证集，使得这三个数据集可以被统一管理。
    train_test_valid_dataset = DatasetDict(
        {
            # 训练集
            'train': train_testvalid['train'],
            # 测试集
            'test': test_valid['test'],
            # 验证集
            'validation': test_valid['train']
        }
    )
    print(f'Dataset after splitting:\n{train_test_valid_dataset}')
    
    # 对数据集进行标记
    print(f'Tokenizing the dataset...')
    tokenized_datasets = train_test_valid_dataset.map(tokenize_function, batched=True)
    # 移除原始数据集的某些列
    tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary'])
    print(f'Tokenizing complete!')
    
    # 创建输出文件夹
    os.makedirs(f'{output_data}/train/', exist_ok=True)
    os.makedirs(f'{output_data}/test/', exist_ok=True)
    os.makedirs(f'{output_data}/validation/', exist_ok=True)
    file_root = str(int(time.time()*1000))
    
    # 将预处理后的数据集写入磁盘
    print(f'Writing the dataset to {output_data}')
    tokenized_datasets['train'].to_parquet(f'./{output_data}/train/{file_root}.parquet')
    tokenized_datasets['test'].to_parquet(f'./{output_data}/test/{file_root}.parquet')
    tokenized_datasets['validation'].to_parquet(f'./{output_data}/validation/{file_root}.parquet')
    print('Preprocessing complete!')
    
'''
定义了两个函数：tokenize_function 和 transform_dataset，它们的主要作用是对输入的数据集进行预处理。

tokenize_function 是一个对数据集样本进行标记（tokenization）的函数。
它接收一个样本作为输入，然后加上特定的提示和结束提示，将对话和摘要信息转换为模型可以理解的标记形式。在这个过程中，对话部分被存储在 'input_ids' 中，摘要部分被存储在 'labels' 中。

transform_dataset 则是一个对数据集进行预处理的函数，包括划分数据集和标记。
首先，它从指定路径加载原始数据集，然后加载适用于特定模型的标记器。
接着，它将数据集划分为训练集、测试集和验证集。之后，对这三个数据集进行标记，将结果存储在新的数据集字典（tokenized_datasets）中。
最后，将这个数据集字典分别写入到训练、测试和验证的文件夹中。

这两个函数的组合使用，可以完成从原始数据集到模型可以直接使用的预处理数据集的转换，包括数据集的划分、样本的标记以及预处理数据的存储。
'''

"\n定义了两个函数：tokenize_function 和 transform_dataset，它们的主要作用是对输入的数据集进行预处理。\n\ntokenize_function 是一个对数据集样本进行标记（tokenization）的函数。它接收一个样本作为输入，然后加上特定的提示和结束提示，将对话和摘要信息转换为模型可以理解的标记形式。在这个过程中，对话部分被存储在 'input_ids' 中，摘要部分被存储在 'labels' 中。\n\ntransform_dataset 则是一个对数据集进行预处理的函数，包括划分数据集和标记。首先，它从指定路径加载原始数据集，然后加载适用于特定模型的标记器。接着，它将数据集划分为训练集、测试集和验证集。之后，对这三个数据集进行标记，将结果存储在新的数据集字典（tokenized_datasets）中。最后，将这个数据集字典分别写入到训练、测试和验证的文件夹中。\n\n这两个函数的组合使用，可以完成从原始数据集到模型可以直接使用的预处理数据集的转换，包括数据集的划分、样本的标记以及预处理数据的存储。\n"

In [13]:
'''
定义函数 process，该函数主要用于处理指定的输入数据，并将结果存储到指定的输出目录。
'''

def process(args):
    # 打印输入数据目录的路径
    print(f"Listing contents of {args.input_data}")
    # 列出输入数据目录下的所有文件
    dirs_input = os.listdir(args.input_data)
    for file in dirs_input:
        print(file)

    # 调用transform_dataset函数对数据进行预处理
    # 包括加载数据、划分数据集、对数据进行标记化、存储处理后的数据
    transform_dataset(input_data=args.input_data,  # 输入数据的路径
                      output_data=args.output_data,  # 输出数据的路径
                      huggingface_model_name=args.model_checkpoint,  # 使用的预训练模型的名字
                      train_split_percentage=args.train_split_percentage,  # 训练集的比例
                      test_split_percentage=args.test_split_percentage,  # 测试集的比例
                      validation_split_percentage=args.validation_split_percentage,  # 验证集的比例
                     )

    # 打印输出数据目录的路径
    print(f"Listing contents of {args.output_data}")
    # 列出输出数据目录下的所有文件
    dirs_output = os.listdir(args.output_data)
    for file in dirs_output:
        print(file)

# 在本地处理自然语言处理（NLP）数据集。

In [14]:
class Args:
    input_data: str  # 输入数据的路径
    output_data: str  # 输出数据的路径
    model_checkpoint: str  # 使用的预训练模型的名字
    train_split_percentage: float  # 训练集的比例
    test_split_percentage: float  # 测试集的比例
    validation_split_percentage: float  # 验证集的比例

args = Args()  # 创建Args类的实例

args.model_checkpoint = model_checkpoint  # 设置模型名字
args.input_data = './data-summarization'  # 设置输入数据的路径
args.output_data = './data-summarization-processed'  # 设置输出数据的路径
args.train_split_percentage = 0.9  # 设置训练集的比例
args.test_split_percentage = 0.05  # 设置测试集的比例
args.validation_split_percentage = 0.05  # 设置验证集的比例

# 如果输出数据的目录已经存在，就删除该目录
if os.path.isdir(args.output_data):
    import shutil
    shutil.rmtree(args.output_data)

# 调用 process 函数对输入数据进行预处理。
process(args)

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/data-summarization-55f5a7efa2dfb458/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


Listing contents of ./data-summarization
dialogsum-2.csv
dialogsum-1.csv


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset loaded from path: ./data-summarization
DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 14460
    })
})
Loading the tokenizer for the model google/flan-t5-base
Dataset after splitting:
DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 13014
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 723
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 723
    })
})
Tokenizing the dataset...


Map:   0%|          | 0/13014 [00:00<?, ? examples/s]

...

Tokenizing complete!
Writing the dataset to ./data-summarization-processed


Creating parquet from Arrow format:   0%|          | 0/14 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Preprocessing complete!
Listing contents of ./data-summarization-processed
test
train
validation


## 确保数据集能够正确加载

In [15]:
'''
使用 load_dataset 函数从指定的路径加载数据集。这里加载的数据集是在前面的预处理步骤中生成的，并已经存储到了指定的目录下。
'''

# 使用load_dataset函数加载数据集
dataset = load_dataset(
    './data-summarization-processed/',  # 数据集所在的路径
    data_files={'train': 'train/*.parquet',  # 训练集文件的路径，这里使用了通配符，表示加载该目录下的所有parquet文件
                'test': 'test/*.parquet',  # 测试集文件的路径
                'validation': 'validation/*.parquet'}  # 验证集文件的路径
)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/data-summarization-processed-501c81c8367fa59b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# 存储变量

In [17]:
local_data_processed_path = './data-summarization-processed/'

In [18]:
%store local_data_processed_path

Stored 'local_data_processed_path' (str)


In [19]:
%store

Stored variables and their in-db values:
ingest_create_athena_table_parquet_passed             -> True
local_data_processed_path                             -> './data-summarization-processed/'
model_checkpoint                                      -> 'google/flan-t5-base'
raw_input_data_s3_uri                                 -> 's3://sagemaker-us-east-1-941797585610/data-summar
role                                                  -> 'arn:aws:iam::941797585610:role/service-role/Amazo
s3_private_path_tsv                                   -> 's3://sagemaker-us-east-1-941797585610/amazon-revi
s3_public_path_tsv                                    -> 's3://dsoaws/tsv'
setup_dependencies_passed                             -> True


# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>